[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Kaggle-runa/MameLand_vol3/blob/main/src/notebook/02_%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E5%89%8D%E5%87%A6%E7%90%86.ipynb)

In [ ]:
# 必要なライブラリのimport
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import OrdinalEncoder

#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 50)

In [35]:
# google driveへのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


データはGoogle Driveの競馬分析/dataレポジトリにあることを想定しています。  
自分のフォルダ構成に応じてデータのパスを適宜変更して下さい。


- 競馬分析/
  - data/  # 分析に使う生データ
  - feature_data/  # 02_データの前処理.ipynbで作成した生データを加工したデータ
  - simulation_data/ # 05_馬券の購入シミュレーション(ワイド・複勝).ipynbで利用する回収率を計算するためのデータ
  - notebooks/  # 競馬分析を行うnotebook
    - 00_データのスクレイピング.ipynb
    - 01_競馬データ可視化.ipynb
    - 02_データの前処理.ipynb
    - 03_モデルの学習.ipynb
    - 04_新規データでの予測.ipynb
    - 05_馬券の購入シミュレーション(ワイド・複勝).ipynb
  - model/  # 作成したモデルを格納するレポジトリ

In [3]:
# データ読み込み
race_result = pd.read_csv('/content/drive/MyDrive/競馬分析/data/race_result.csv')

In [4]:
# データの表示
race_result.head()

,race_id,event_date,location,race_title,race_type,race_turn,course_len,weather,ground_condition,finish_position,frame_number,horse_number,horse_id,horse_name,sex_age,carried_weight,jockey_id,jockey,time,difference,odds,popularity,horse_weight,trainer
0,202206010101,2022-01-05,中山,3歳未勝利,ダ,右,1200,晴,良,1,8,15,2019103610,ニシノアナ,牝3,51.0,1192,横山琉人,1:12.5,NaN,6.8,4.0,456(+4),[東] 相沢郁
1,202206010101,2022-01-05,中山,3歳未勝利,ダ,右,1200,晴,良,2,5,10,2019100855,トラストパッキャオ,牝3,54.0,1179,菅原明良,1:12.5,クビ,57.2,12.0,458(+2),[東] 高木登
2,202206010101,2022-01-05,中山,3歳未勝利,ダ,右,1200,晴,良,3,2,4,2019103542,マイネルシトラス,牡3,56.0,1009,柴田大知,1:12.5,クビ,3.7,1.0,518(-2),[東] 武市康男
3,202206010101,2022-01-05,中山,3歳未勝利,ダ,右,1200,晴,良,4,1,2,2019104288,ピカリエ,牝3,54.0,1119,伊藤工真,1:12.8,1.1/2,16.0,9.0,486(+6),[東] 金成貴史
4,202206010101,2022-01-05,中山,3歳未勝利,ダ,右,1200,晴,良,5,8,16,2019101003,ブラッドライン,牡3,56.0,5212,Ｍ．デム,1:13.2,2.1/2,10.0,5.0,478(-2),[東] 伊藤大士


In [5]:
# 性齢
# 馬の年齢と性別が１つのカラムにまとめられているので2つのカラムに分割する
race_result_sex = race_result['sex_age'].str.extract('([牝牡セ])(\d+)', expand=True)

# 新規カラムの作成
race_result['sex'] = race_result_sex.loc[:, 0]
race_result['age'] = race_result_sex.loc[:, 1].astype(int)

# 性齢のカラムを削除
race_result = race_result.drop(['sex_age'],axis=1)

In [6]:
# 馬体重(増減)
# 馬の体重と前回のレースからの増減幅が１つのカラムにまとめられているので2つのカラムに分割する
race_result_weight = race_result['horse_weight'].str.extract('(\d{3}).([+-0]\d*)', expand=True)

# 新規カラムの作成
race_result['horse_weight'] = race_result_weight.loc[:, 0].fillna(0).astype(int)
race_result['weight_gain_loss'] = race_result_weight.loc[:, 1].str.replace('\+', '', regex=True).fillna(0).astype(int)

#馬体重（増減）のカラムを削除
race_result = race_result.drop(['horse_weight'], axis=1)

In [7]:
# 調教師
# 調教師の名前と東西が１つのカラムにまとめられているため２つのカラムに分割する
race_result_trainner = race_result['trainer'].str.extract(r'\[(.)\] (.+)', expand=True)

# 新しいカラムの作成
race_result['trainer_region'] = race_result_trainner.loc[:, 0]
race_result['trainer_name'] = race_result_trainner.loc[:, 1]

race_result = race_result.drop(['trainer'], axis=1)

In [8]:
# レースの出走頭数を算出
# 取消・除外になっている馬はレースに走っていないのでカウントから外す

# finish_positionが「取」または「除」でないレコードのみをカウント
df_filtered = race_result[~race_result['finish_position'].isin(['取', '除'])]

# race_idごとに出走頭数をカウント
head_count = df_filtered.groupby('race_id').size().reset_index(name='horse_count')

# 元のデータフレームにhead_countをマージ
race_result = race_result.merge(head_count, on='race_id', how='left')

In [9]:
# race_titleに基づいてrace_gradeカラムを追加
grade_list = ['新馬', '未勝利', '1勝', '2勝', '3勝', 'OP', 'L', 'GI', 'GII', 'GIII', 'JGI', 'JGII', 'JGIII', 'オープン']

def get_race_grade(title):
    for grade in grade_list:
        if grade in title:
            if grade == 'オープン':
                return '障害オープン'
            return grade
    return None

race_result['race_grade'] = race_result['race_title'].apply(get_race_grade)

In [10]:
# オッズの処理
# 取消・除外になっている馬のオッズが「---」となっているため、明らかに異常値であることを示す999に変換する
race_result['odds'] = race_result['odds'].replace('---', 999).astype(float)

In [11]:
# 着順の処理
# 取消・除外・取消になっているものを、明らかに異常値であることを示す999に変換する
race_result['finish_position'] = (race_result['finish_position'] .replace('中', 999) .replace('除', 999) .replace('取', 999) .replace(r'(\d+)\(降\)', r'\1', regex=True) .astype(float))

In [12]:
# 余計な列の削除
# horse_nameとhorse_id、jockeyとjockey_idは同じ情報を示しているため片方を削除しておく
race_result = race_result.drop(['horse_name','jockey'], axis=1)

# timeとdifference(着差)は予測時には分からないデータになるので一旦削除する
race_result = race_result.drop(['time','difference'], axis=1)

In [13]:
# 日付の処理
# race_ymd 2015-08-01を 2015 8 1に分割してintに変換する
race_result[['year', 'month', 'day']] = race_result['event_date'].str.split('-', expand=True).astype(int)

# 後々日付を元にデータの並び替えをするために変換
race_result['event_date'] = pd.to_datetime(race_result['event_date'])

In [17]:
# 残りのobject型をOrdinalEncoderにかける
# エンコーディングの種類：https://thefinance.jp/tecnology/201109-2

# カテゴリー列のみ取得
categorical_columns = list(race_result.select_dtypes(include=object).columns)

# カテゴリカル変数を文字列に変換
race_result[categorical_columns] = race_result[categorical_columns].astype(str)

# 未知の値は -1 に変換する
ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

# カテゴリカル変数のみをエンコード
ordinal_encoder.fit(race_result[categorical_columns])

ordinal_encoder.fit(race_result[categorical_columns])
race_result[categorical_columns] = ordinal_encoder.transform(race_result[categorical_columns])

# OrdinalEncoderの結果の保存
joblib.dump(ordinal_encoder, "/content/drive/MyDrive/競馬分析/model/ordinal_encoder.pkl")

In [15]:
# 目的変数の作成
# ３着以内に入った馬に1、それ以外は0のフラグを立てる
race_result['target'] = (race_result['finish_position'] <= 3).astype(int)

In [16]:
# データの並び替え
# 元々、データはrace_idごとにfinish_position（着順）の昇順で並んでいたが、そのままではAIがその傾向を捉えてしまい、正確な分析ができない可能性があるためデータをhorse_number（馬番）で並び替える。
race_result = race_result.sort_values(by=['event_date', 'race_id', 'horse_number'], ascending=[True, True, True])
race_result

,race_id,event_date,location,race_title,race_type,race_turn,course_len,weather,ground_condition,finish_position,frame_number,horse_number,horse_id,carried_weight,jockey_id,odds,popularity,sex,age,weight_gain_loss,trainer_region,trainer_name,horse_count,race_grade,year,month,day,target
10,202206010101,2022-01-05,1.0,9.0,0.0,0.0,1200,2.0,2.0,11.0,1,1,2019102173,51.0,1177,258.1,15.0,1.0,3,6,2.0,108.0,16,7.0,2022,1,5,0
3,202206010101,2022-01-05,1.0,9.0,0.0,0.0,1200,2.0,2.0,4.0,1,2,2019104288,54.0,1119,16.0,9.0,1.0,3,6,2.0,256.0,16,7.0,2022,1,5,0
15,202206010101,2022-01-05,1.0,9.0,0.0,0.0,1200,2.0,2.0,16.0,2,3,2019106127,54.0,1178,32.8,11.0,1.0,3,4,2.0,169.0,16,7.0,2022,1,5,0
2,202206010101,2022-01-05,1.0,9.0,0.0,0.0,1200,2.0,2.0,3.0,2,4,2019103542,56.0,1009,3.7,1.0,2.0,3,-2,2.0,165.0,16,7.0,2022,1,5,1
12,202206010101,2022-01-05,1.0,9.0,0.0,0.0,1200,2.0,2.0,13.0,3,5,2019101943,54.0,1029,273.2,16.0,1.0,3,-2,2.0,182.0,16,7.0,2022,1,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132280,202407030912,2024-09-29,0.0,7.0,0.0,1.0,1400,3.0,1.0,11.0,6,12,2018100421,53.0,1212,48.5,11.0,1.0,6,2,3.0,112.0,16,1.0,2024,9,29,0
132285,202407030912,2024-09-29,0.0,7.0,0.0,1.0,1400,3.0,1.0,16.0,7,13,2017100447,58.0,1138,219.3,16.0,2.0,7,0,3.0,120.0,16,1.0,2024,9,29,0
132284,202407030912,2024-09-29,0.0,7.0,0.0,1.0,1400,3.0,1.0,15.0,7,14,2019109138,57.0,1200,110.7,13.0,0.0,5,6,2.0,137.0,16,1.0,2024,9,29,0
132278,202407030912,2024-09-29,0.0,7.0,0.0,1.0,1400,3.0,1.0,9.0,8,15,2020103419,57.0,1208,8.2,5.0,2.0,4,-4,3.0,139.0,16,1.0,2024,9,29,0


In [18]:
# データの保存
race_result.to_csv('/content/drive/MyDrive/競馬分析/feature_data/feature_race_result.csv', index=None)